## Get ground truth from active model
\*PLEASE NOTE YOU NEED TO COMPILE THE COMPILED/MECHANISMS FOLDER BEFORE RUNNING\*

In [1]:
import os, sys
sys.path.append(os.path.split(sys.path[0])[0])

#Dependencies
import h5py
from neuron import h
import numpy as np
import pandas as pd
from scipy import signal

#Project Imports
import config.params as params
import config.paths as paths
# from stylized_module.base.active_model_synapse_input import *
from stylized_module.base.passive_model_soma_injection import *

# h.nrn_load_dll(os.path.join(mechanisms_path,'nrnmech.dll'))  # running this line more than once might crash the kernel
# h.nrn_load_dll(os.path.join(mechanisms_path,'x86_64/.libs/libnrnmech.so'))  # use this line instead in Linux
h.nrn_load_dll(paths.COMPILED_LIBRARY)
geo_standard = pd.read_csv(paths.GEO_STANDARD,index_col='id')
h.tstop = params.GT_TSTOP
h.dt = params.GT_DT

Build the simulation, generate an LFP, filter the LFP, and save it to an h5 file

In [2]:
sim = Simulation(
    geo_standard,
    params.GT_ELECTRODE_POSITION,
    params.GT_LOCATION_PARAMETERS,
#     gmax=params.GT_GMAX,
    scale=params.GT_SCALE)
sim.run()
lfp = sim.get_lfp().T

filt_b,filt_a = signal.butter(params.GT_BUTTERWORTH_ORDER,
                              params.GT_CRITICAL_FREQUENCY,
                              params.GT_BANDFILTER_TYPE,
                              fs=params.GT_FILTER_SAMPLING_RATE)

lfp = signal.lfilter(filt_b,filt_a,lfp,axis=0)

hf = h5py.File(paths.SIMULATED_DATA_FILE, 'w')
hf.create_dataset('data',data=lfp)
hf.close()